In [ ]:
!pip install pynrrd

In [ ]:
from tqdm import tqdm
import os
from random import randint

import numpy as np
import pandas as pd

import nibabel as nib
import pydicom as pdm
import nilearn as nl
import nilearn.plotting as nlplt
import nrrd
import h5py

import matplotlib.pyplot as plt
from matplotlib import cm
import matplotlib.animation as anim

import imageio
from skimage.transform import resize
from skimage.util import montage

from IPython.display import Image as show_gif

import warnings
warnings.simplefilter("ignore")

In [ ]:
class ImageToGIF:
    """Create GIF without saving image files."""
    def __init__(self,
                 size=(600, 400), 
                 xy_text=(80, 10),
                 dpi=100, 
                 cmap='CMRmap'):

        self.fig = plt.figure()
        self.fig.set_size_inches(size[0] / dpi, size[1] / dpi)
        self.xy_text = xy_text
        self.cmap = cmap
        
        self.ax = self.fig.add_axes([0, 0, 1, 1])
        self.ax.set_xticks([])
        self.ax.set_yticks([])
        self.images = []
 
    def add(self, *args, label, with_mask=True):
        
        image = args[0]
        mask = args[-1]
        plt.set_cmap(self.cmap)
        plt_img = self.ax.imshow(image, animated=True)
        if with_mask:
            plt_mask = self.ax.imshow(np.ma.masked_where(mask == False, mask),
                                      alpha=0.7, animated=True)

        plt_text = self.ax.text(*self.xy_text, label, color='red')
        to_plot = [plt_img, plt_mask, plt_text] if with_mask else [plt_img, plt_text]
        self.images.append(to_plot)
        plt.close()
 
    def save(self, filename, fps):
        animation = anim.ArtistAnimation(self.fig, self.images)
        animation.save(filename, writer='imagemagick', fps=fps)
        
        
class Image3dToGIF3d:
    """
    Displaying 3D images in 3d axes.
    Parameters:
        img_dim: shape of cube for resizing.
        figsize: figure size for plotting in inches.
    """
    def __init__(self, 
                 img_dim: tuple = (55, 55, 55),
                 figsize: tuple = (15, 10),
                ):
        """Initialization."""
        self.img_dim = img_dim
        print(img_dim)
        self.figsize = figsize
    
    def _explode(self, data: np.ndarray):
        """
        Takes: array and return an array twice as large in each dimension,
        with an extra space between each voxel.
        """
        shape_arr = np.array(data.shape)
        size = shape_arr[:3] * 2 - 1
        exploded = np.zeros(np.concatenate([size, shape_arr[3:]]),
                            dtype=data.dtype)
        exploded[::2, ::2, ::2] = data
        return exploded

    def _expand_coordinates(self, indices: np.ndarray):
        x, y, z = indices
        x[1::2, :, :] += 1
        y[:, 1::2, :] += 1
        z[:, :, 1::2] += 1
        return x, y, z
    
    def _normalize(self, arr: np.ndarray):
        """Normilize image value between 0 and 1."""
        return arr / arr.max()
    
    def _scale_by(self, arr: np.ndarray, factor: int):
        """
        Scale 3d Image to factor.
        Parameters:
            arr: 3d image for scalling.
            factor: factor for scalling.
        """
        mean = np.mean(arr)
        return (arr - mean) * factor + mean
    
    def get_transformed_data(self, data: np.ndarray):
        """Data transformation: normalization, scaling, resizing."""
        norm_data = np.clip(self._normalize(data)-0.1, 0, 1) ** 0.4
        scaled_data = np.clip(self._scale_by(norm_data, 2) - 0.1, 0, 1)
        resized_data = resize(scaled_data, self.img_dim, mode='constant')
        return resized_data
    
    def plot_cube(self,
                  cube,
                  title: str = '', 
                  init_angle: int = 0,
                  make_gif: bool = False,
                  path_to_save: str = 'filename.gif'
                 ):
        """
        Plot 3d data.
        Parameters:
            cube: 3d data
            title: title for figure.
            init_angle: angle for image plot (from 0-360).
            make_gif: if True create gif from every 5th frames from 3d image plot.
            path_to_save: path to save GIF file.
            """
        cube = self._normalize(cube)

        facecolors = cm.gist_stern(cube)
        facecolors[:,:,:,-1] = cube
        facecolors = self._explode(facecolors)

        filled = facecolors[:,:,:,-1] != 0
        x, y, z = self._expand_coordinates(np.indices(np.array(filled.shape) + 1))

        with plt.style.context("dark_background"):

            fig = plt.figure(figsize=self.figsize)
            ax = fig.gca(projection='3d')

            ax.view_init(30, init_angle)
            ax.set_xlim(right = self.img_dim[0] * 2)
            ax.set_ylim(top = self.img_dim[1] * 2)
            ax.set_zlim(top = self.img_dim[2] * 2)
            ax.set_title(title, fontsize=18, y=1.05)

            ax.voxels(x, y, z, filled, facecolors=facecolors, shade=False)

            if make_gif:
                images = []
                for angle in tqdm(range(0, 360, 5)):
                    ax.view_init(30, angle)
                    fname = str(angle) + '.png'

                    plt.savefig(fname, dpi=120, format='png', bbox_inches='tight')
                    images.append(imageio.imread(fname))
                    #os.remove(fname)
                imageio.mimsave(path_to_save, images)
                plt.close()

            else:
                plt.show()

# Nifti File

In [ ]:
sample_filename = '../input/brats20-dataset-training-validation/BraTS2020_TrainingData/MICCAI_BraTS2020_TrainingData/BraTS20_Training_001/BraTS20_Training_001_flair.nii'
sample_filename_mask = '../input/brats20-dataset-training-validation/BraTS2020_TrainingData/MICCAI_BraTS2020_TrainingData/BraTS20_Training_001/BraTS20_Training_001_seg.nii'

sample_img = nib.load(sample_filename)
sample_img = np.asanyarray(sample_img.dataobj)
sample_mask = nib.load(sample_filename_mask)
sample_mask = np.asanyarray(sample_mask.dataobj)
print("img shape ->", sample_img.shape)
print("mask shape ->", sample_mask.shape)

visualization of 3d data in 2d slices

In [ ]:
slice_n = 100
fig, ax = plt.subplots(2, 3, figsize=(25, 15))

ax[0, 0].imshow(sample_img[slice_n, :, :])
ax[0, 0].set_title(f"image slice number {slice_n} along the x-axis", fontsize=18, color="red")
ax[1, 0].imshow(sample_mask[slice_n, :, :])
ax[1, 0].set_title(f"mask slice {slice_n} along the x-axis", fontsize=18, color="red")

ax[0, 1].imshow(sample_img[:, slice_n, :])
ax[0, 1].set_title(f"image slice number {slice_n} along the y-axis", fontsize=18, color="red")
ax[1, 1].imshow(sample_mask[:, slice_n, :])
ax[1, 1].set_title(f"mask slice number {slice_n} along the y-axis", fontsize=18, color="red")

ax[0, 2].imshow(sample_img[:, :, slice_n])
ax[0, 2].set_title(f"image slice number {slice_n} along the z-axis", fontsize=18, color="red")
ax[1, 2].imshow(sample_mask[:, :, slice_n])
ax[1, 2].set_title(f"mask slice number {slice_n}along the z-axis", fontsize=18, color="red")
fig.tight_layout()
plt.show()

In [ ]:
## matching colormaps
#Greys_r RdGy_r  CMRmap afmhot binary_r bone copper cubehelix gist_heat gist_stern gnuplot hot inferno magma nipy_spectral

sample_data_gif = ImageToGIF()
label = sample_filename.replace('/', '.').split('.')[-2]
filename = f'{label}_3d_2d.gif'

for i in range(sample_img.shape[0]):
    image = np.rot90(sample_img[i])
    mask = np.clip(np.rot90(sample_mask[i]), 0, 1)
    sample_data_gif.add(image, mask, label=f'{label}_{str(i)}')
 
sample_data_gif.save(filename, fps=15)
show_gif(filename, format='png')

In [ ]:
image = np.rot90(montage(sample_img))
mask = np.rot90(montage(sample_mask)) 
mask = np.clip(mask, 0, 1)

fig, ax1 = plt.subplots(1, 1, figsize = (20, 20))
ax1.imshow(image, cmap ='bone')
ax1.imshow(np.ma.masked_where(mask == False, mask),
           cmap='cool', alpha=0.6, animated=True)
fig.savefig(f'{label}_3d_to_2d.png', format='png', bbox_inches='tight', pad_iches=0.0)

and 3d

In [ ]:
%%time
title = sample_filename.replace(".", "/").split("/")[-2]
filename = title+"_3d.gif"

data_to_3dgif = Image3dToGIF3d()#img_dim = (120, 120, 78)
transformed_data = data_to_3dgif.get_transformed_data(sample_img)
data_to_3dgif.plot_cube(
    transformed_data[:38, :47, :35],#[:77, :105, :55]
    title=title,
    make_gif=True,
    path_to_save=filename
)
show_gif(filename, format='png')

visualization with nilearn

In [ ]:
niimg = nl.image.load_img(sample_filename)
nimask = nl.image.load_img(sample_filename_mask)

In [ ]:
fig, axes = plt.subplots(nrows=4, figsize=(30, 40))


nlplt.plot_anat(niimg,
                title='BraTS20_Training_001_flair.nii plot_anat',
                axes=axes[0])

nlplt.plot_epi(niimg,
               title='BraTS20_Training_001_flair.nii plot_epi',
               axes=axes[1])

nlplt.plot_img(niimg,
               title='BraTS20_Training_001_flair.nii plot_img',
               axes=axes[2])

nlplt.plot_roi(nimask, 
               title='BraTS20_Training_001_flair.nii with mask plot_roi',
               bg_img=niimg, 
               axes=axes[3], cmap='Paired')

plt.show()

In [ ]:
t1_niimg  = nl.image.load_img('../input/brats20-dataset-training-validation/BraTS2020_TrainingData/MICCAI_BraTS2020_TrainingData/BraTS20_Training_001/BraTS20_Training_001_t1.nii')
t2_niimg  = nl.image.load_img('../input/brats20-dataset-training-validation/BraTS2020_TrainingData/MICCAI_BraTS2020_TrainingData/BraTS20_Training_001/BraTS20_Training_001_t2.nii')

fig, axes = plt.subplots(nrows=2, figsize=(30, 20))

nlplt.plot_epi(t1_niimg, title="BraTS20_Training_001_t1.nii plot_epi", axes=axes[0])
nlplt.plot_epi(t2_niimg, title="BraTS20_Training_001_t2.nii plot_epi", axes=axes[1])
plt.show()

# DICOM File

In [ ]:
sample_path = '../input/osic-pulmonary-fibrosis-progression/train/ID00015637202177877247924'
sample_path_files = sorted(os.listdir(sample_path), key=lambda x: int(x[:-4]))

visualization of 3d data in 2d slices

In [ ]:
sample_data_gif = ImageToGIF(size=(768, 768),
                             xy_text=(250, 15))

label = sample_path.split('/')[-1]
for i in range(len(sample_path_files)):
    path = os.path.join(sample_path, sample_path_files[i])
    image = pdm.dcmread(path).pixel_array
    sample_data_gif.add(image, label=f'{label}_{str(i)}', with_mask=False)
 
sample_data_gif.save(f'{label}.gif', fps=15)
show_gif(f'{label}.gif', format='png')

# Nrrd File

In [ ]:
def read_nrrd_file(path: str, 
                   tensor_shape: tuple ) -> np.ndarray:
    if os.path.exists(path):
        tensor = nrrd.read(path)[0]                             
        tensor = np.flip(tensor, -1)                   # Warning! slice order of images and masks does not match.
    else: 
        tensor = np.zeros(tensor_shape, dtype=np.float32)
    return tensor


def nrrd_to_numpy(id_: str, tensor_shape: tuple):
    '''
    Returns:  all id masks in single numpy tensor.
    '''
    lung_file_path = '../input/ct-lung-heart-trachea-segmentation/nrrd_lung/nrrd_lung/' + id_ + '_lung.nrrd'
    heart_file_path  = '../input/ct-lung-heart-trachea-segmentation/nrrd_heart/nrrd_heart/' + id_ + '_heart.nrrd'         # Here path hardcoded
    trachea_file_path = '../input/ct-lung-heart-trachea-segmentation/nrrd_trachea/nrrd_trachea/' + id_ + '_trachea.nrrd'

    lung_tensor = read_nrrd_file(lung_file_path, tensor_shape)
    heart_tensor = read_nrrd_file(heart_file_path, tensor_shape)
    trachea_tensor = read_nrrd_file(trachea_file_path, tensor_shape)
    
    # chek if all tensors  have the same shape.
    if not (lung_tensor.shape == heart_tensor.shape == trachea_tensor.shape):
        #print(lung_tensor.shape, heart_tensor.shape, trachea_tensor.shape)
        print("problem with id:", id_)
        return 
        
    # now each tensor channel is a mask with a unique label
    full_mask = np.stack([lung_tensor, heart_tensor, trachea_tensor])

    # reorient the axes from CHWB to BWHC
    full_mask = np.moveaxis(full_mask,
                            [0, 1, 2, 3],
                            [3, 2, 1, 0]).astype(np.float32)

    return full_mask

In [ ]:
id_ = 'ID00015637202177877247924'

sample_masks = nrrd_to_numpy(id_, (768, 768))
sample_masks.shape

visualization of 3d data in 2d slices

In [ ]:
sample_data_gif = ImageToGIF(size=(768, 768),
                             xy_text=(250, 15))

label = sample_path.split('/')[-1] + '_mask'
for i in range(sample_masks.shape[0]):
    sample_data_gif.add(sample_masks[i],label=f'{label}_{str(i)}', with_mask=False)
 
sample_data_gif.save(f'{label}.gif', fps=15)
show_gif(f'{label}.gif', format='png')

and images with masks

In [ ]:
class ImageToGIF:
    """Create GIF without saving image files."""
    def __init__(self,
                 size=(768, 768), 
                 xy_text=(250, 15),
                 dpi=100):

        self.fig = plt.figure()
        self.fig.set_size_inches(size[0] / dpi, size[1] / dpi)
        self.xy_text = xy_text
        
        self.ax = self.fig.add_axes([0, 0, 1, 1])
        self.ax.set_xticks([])
        self.ax.set_yticks([])
        self.images = []
 
    def add(self, image, mask, label, with_mask=True):
        
        lung, heart, trachea = [mask[:, :, i] for i in range(3)]
        plt_img = self.ax.imshow(image, cmap="bone", animated=True)

        # Overlaying segmentation masks
        plt_mask1 = self.ax.imshow(np.ma.masked_where(lung == False, lung),
                               cmap='cool', alpha=0.3, animated=True)
        plt_mask2 = self.ax.imshow(np.ma.masked_where(heart == False, heart),
                               cmap='autumn', alpha=0.3, animated=True)
        plt_mask3 = self.ax.imshow(np.ma.masked_where(trachea == False, trachea),
                               cmap='autumn_r', alpha=0.3, animated=True) #cool_r - blue
        plt_text = self.ax.text(*self.xy_text, label, color='red')
        to_plot = [plt_img, plt_mask1, plt_mask2, plt_mask3, plt_text] 
        self.images.append(to_plot)
        plt.close()
 
    def save(self, filename, fps):
        animation = anim.ArtistAnimation(self.fig, self.images)
        animation.save(filename, writer='imagemagick', fps=fps)

In [ ]:
sample_data_gif = ImageToGIF()

label = sample_path.split('/')[-1] + '_with_masks'
for i in range(sample_masks.shape[0]):
    path = os.path.join(sample_path, sample_path_files[i])
    image = pdm.dcmread(path).pixel_array
    mask = sample_masks[i]
    sample_data_gif.add(image, mask, label=f'{label}_{str(i)}',)
 
sample_data_gif.save(f'{label}.gif', fps=15)
show_gif(f'{label}.gif', format='png')

# MATLAB File

In [ ]:
sample_filename = '../input/trends-assessment-prediction/fMRI_train/10001.mat'
matlab_file = h5py.File(sample_filename)
print(matlab_file.keys())
print(matlab_file.values())
print(matlab_file['SM_feature'][()].shape)

In [ ]:
def get_spatial_maps(path: str) -> np.ndarray:
    """
    Read the given .mat file with h5py
    Switch second and fourth axes (53, 52, 63, 53) -> (53, 53, 63, 52)
    Make identical ZYX axes (53, 53, 63, 52) -> (53, 63, 63, 63)
    
    Params:
        path - of the MATLAB file
    
    Returns:
        mew_spatial_maps - nympy array, shape = (53, 63, 63, 63)
        masked_spatial_data : numpy array, shape = (53, 58869)
        numpy array after filling ZYX dimensions.
    """
    matlab_file = h5py.File(path)

    spatial_maps = np.moveaxis(matlab_file['SM_feature'][()], [0, 1, 2, 3], [3, 2, 1, 0])


    max_lengh_ax = np.argmax([spatial_maps.shape[0], spatial_maps.shape[1], spatial_maps.shape[2]])
    new_spatial_maps = np.full((*[spatial_maps.shape[max_lengh_ax]]*3, spatial_maps.shape[-1],), 0, dtype='<f8')

    new_spatial_maps[:spatial_maps.shape[0],
                     :spatial_maps.shape[1],
                     :spatial_maps.shape[2],
                     :spatial_maps.shape[3]] = spatial_maps
    
    return new_spatial_maps

In [ ]:
sample_spatial_maps_file = '../input/trends-assessment-prediction/fMRI_train/10009.mat'
sample_mask_file = '../input/trends-assessment-prediction/fMRI_mask.nii'
gifname = sample_spatial_maps_file.replace('.', '/').split("/")[-2]

fmri_mask = nl.image.load_img(sample_mask_file)
fmri_mask = np.asarray(fmri_mask.dataobj)
spatial_maps = get_spatial_maps(sample_spatial_maps_file)

print("spatial_maps ->", spatial_maps.shape)

visualization of 3d data in 2d slices

In [ ]:
PATH_TO_SAVE = "test"
if not os.path.exists(PATH_TO_SAVE):
    os.mkdir(PATH_TO_SAVE)
    print(f'Folder {PATH_TO_SAVE} created.')

In [ ]:
num_slices = spatial_maps.shape[0]
step = randint(0, 52)

fig, ax = plt.subplots(nrows=1, ncols=3, figsize=(20, 12.5))
plt.set_cmap('bwr')
[axi.set_axis_off() for axi in ax.ravel()]

for slice_ in np.arange(num_slices):
    ax[0].imshow(np.rot90(spatial_maps[slice_, :, :, step], axes=(-2,-1)))
    ax[1].imshow(np.rot90(spatial_maps[:, slice_, :, step], axes=(-2,-1)))
    ax[2].imshow(np.rot90(spatial_maps[:, :, slice_, step], axes=(-2,-1)))

    ax[0].set_title('X-axis {}'.format(slice_), fontsize=15,weight='bold', color='white')
    ax[1].set_title('Y-axis {}'.format(slice_), fontsize=15, weight='bold', color='white')
    ax[2].set_title('Z-axis {}'.format(slice_), fontsize=15, weight='bold', color='white')

    plt.subplots_adjust(wspace=0, hspace=0)
    fig.savefig(f'{PATH_TO_SAVE}/{slice_}.png', format='png', bbox_inches='tight', pad_iches=0.0, facecolor='#0e1111') 
    plt.close()

In [ ]:
images = []
filenames = sorted(os.listdir('test'), key=lambda x: int(x[:-4]))
for filename in filenames:
    filename = 'test/' + filename
    images.append(imageio.imread(filename))
imageio.mimsave(f'{gifname}.gif', images, duration=0.5)

show_gif(f'{gifname}.gif', format='png')

In [ ]:
!rm -r test

and 3d 

In [ ]:
spatial_maps = matlab_file['SM_feature'][()]
n_map = 22
title = sample_filename.split("/")[-1]
path_to_save = title + "_3d.gif"

X,Y,Z = [], [], [] 
roi = []

for z, image in enumerate(spatial_maps[n_map,:,:,:]):
    xx, yy = np.meshgrid(np.linspace(0, image.shape[1],image.shape[1]),
                         np.linspace(0, image.shape[0],image.shape[0]))

    zz = np.ones(xx.shape) * z
    xx = xx[[image != 0][0]]
    yy = yy[[image != 0][0]]
    zz = np.ones(xx.shape) * z
    X += list(xx)
    Y += list(yy)
    Z += list(zz)
    roi += list(image[[image != 0][0]])

fig = plt.figure(figsize=(12, 10))
ax = fig.gca(projection='3d')
ax.set_title(title, fontsize=15, y=1.01)
ax.scatter(X, Y, Z, c = plt.cm.bwr(roi), cmap ='bwr', s=1, alpha=1)

images = []
for angle in tqdm(range(0, 360, 5)):
    ax.view_init(30, angle)
    fname = str(angle) + '.png'
    #print(fname)
    plt.savefig(fname, dpi=120, format='png', bbox_inches='tight')
    images.append(imageio.imread(fname))
    #os.remove(fname)
imageio.mimsave(path_to_save, images)
plt.close()

show_gif(path_to_save)

visualization using nilearn 

In [ ]:
!wget https://github.com/Chaogan-Yan/DPABI/raw/master/Templates/ch2better.nii

In [ ]:
# Loading reference image
fmri_mask = nl.image.load_img('../input/trends-assessment-prediction/fMRI_mask.nii')

# Reorienting the axis of 3D spatial map
spatial_maps = np.moveaxis(matlab_file['SM_feature'][()], [0, 1, 2, 3], [3, 2, 1, 0]) 

# Loading 3D spatial maps
spatial_maps_niimg = nl.image.new_img_like(ref_niimg=fmri_mask,
                                           data=spatial_maps,
                                           affine=fmri_mask.affine,
                                           copy_header=True)

In [ ]:
PATH_TO_SAVE = "test"
if not os.path.exists(PATH_TO_SAVE):
    os.mkdir(PATH_TO_SAVE)
    print(f'Folder {PATH_TO_SAVE} created.')

In [ ]:
for i, img in enumerate(list(nl.image.iter_img(spatial_maps_niimg))):
    fig, ax = plt.subplots(figsize=(18, 5.5))
    nlplt.plot_stat_map(stat_map_img=img,
                bg_img='ch2better.nii',
                title=f'10001.mat Spatial Map {str(i)} plot_stat_map',
                axes=ax,
                threshold=1,
                display_mode='ortho',
                annotate=False,
                draw_cross=True,
                colorbar=False)
    plt.subplots_adjust(wspace=0, hspace=0)
    fig.savefig(f'{PATH_TO_SAVE}/{i}.png', format='png', bbox_inches='tight', pad_iches=0.0, facecolor='#0e1111') 
    plt.close()

In [ ]:
images = []
filenames = sorted(os.listdir('test'), key=lambda x: int(x[:-4]))
for filename in filenames:
    filename = 'test/' + filename
    images.append(imageio.imread(filename))
imageio.mimsave(f'{gifname}_nil.gif', images, duration=0.5)

show_gif(f'{gifname}_nil.gif', format='png')

In [ ]:
!rm -r test

and 3d

In [ ]:
img = img = list(nl.image.iter_img(spatial_maps_niimg))[0]
view = nlplt.view_img_on_surf(img,
                              title=f'10009.mat Spatial Map 0 view_img_on_surf',
                              title_fontsize=20,
                              threshold=1,
                              black_bg=False)
view.open_in_browser()
view

References:
+ https://nilearn.github.io/auto_examples/index.html
+ https://terbium.io/2017/12/matplotlib-3d/
+ https://tomroelandts.com/articles/how-to-create-animated-gifs-with-python
+ https://www.kaggle.com/gunesevitan/trends-neuroimaging-data-analysis-3d-features#5.-Component-Spatial-Maps
+ https://www.kaggle.com/mawanda/fmri-3d-brain-images-dynamic-drawing